kmp 2022

Closely based on 

**Ben Lauwens & Allen Downey "Think Julia: How to Think Like a Computer Scientist" 
https://benlauwens.github.io/ThinkJulia.jl/latest/book.html**

Resources:

Julia webpage https://julialang.org/ 

Julia documentation https://docs.julialang.org/en/v1/


In [1]:
using Pkg
Pkg.add("Unicode")
using Unicode

    Updating registry at `C:\Users\kmpetersson\.julia\registries\General.toml`
   Resolving package versions...
    Updating `C:\Users\kmpetersson\.julia\environments\v1.7\Project.toml`
  [4ec0a83e] + Unicode
  No Changes to `C:\Users\kmpetersson\.julia\environments\v1.7\Manifest.toml`


## Chapter 08 -- Strings

A **string** is a sequence of characters, or Chars, which means it is an **ordered collection of other character values** and therefore a so-called *iterable**. Char is a datatype of its own.

### Characters

English language speakers are familiar with **characters** such as the letters of the alphabet (A, B, C,…), numerals, and common punctuation symbols. 

These characters are standardized and mapped to integer values between 0 and 127 by the **ASCII standard** (American Standard Code for Information Interchange).

In [3]:
Int('A'):Int('Z')

65:90

In [4]:
Int('a'):Int('z')

97:122

In [5]:
Char(93)

']': ASCII/Unicode U+005D (category Pe: Punctuation, close)

There are many other characters used in non-English languages, including variants of the ASCII characters with accents and other modifications, related scripts such as Cyrillic and Greek, and scripts completely unrelated to ASCII and English, including Arabic, Chinese, Hebrew, Hindi, Japanese, and Korean.

The **Unicode standard** (https://en.wikipedia.org/wiki/Unicode) tackles the complexities of what exactly a character is, and is generally accepted as the definitive standard addressing this problem. It provides a unique number for every character on a world-wide scale.

A **Char** represents a single character and is surrounded by single quotes:

```Julia
	julia> 'x'
		'x': ASCII/Unicode U+0078 (category Ll: Letter, lowercase)
	julia> '🍌'
		'🍌': Unicode U+01f34c (category So: Symbol, other)
	julia> typeof('x')
		Char
```

Also emojis are part of the Unicode standard.

In [7]:
'x'

'x': ASCII/Unicode U+0078 (category Ll: Letter, lowercase)

In [8]:
'🍌'    # ' \:banana:<TAB> '

'🍌': Unicode U+1F34C (category So: Symbol, other)

In [9]:
typeof('🍌')

Char

In [12]:
🐫 = 3.5

3.5

In [13]:
typeof(🐫)

Float64

### A string is a sequence of characters

A _`string`_ is a sequence of characters, indicated by **"string"**, and you can access these characters one at a time with the **bracket operator**:

```Julia
	julia> fruit = "banana"
		"banana"
	julia> letter = fruit[1]
		'b': ASCII/Unicode U+0062 (category Ll: Letter, lowercase)
```

The second statement selects character number 1 from fruit and assigns it to letter. The expression in brackets is called an **index**. The index indicates which character in the sequence you want.

All indexing in Julia is 1-based: the first element of any integer-indexed object is found at index 1 and the last element at **index end**:

In [14]:
fruit = "banana"
fruit[begin]

'b': ASCII/Unicode U+0062 (category Ll: Letter, lowercase)

### Index expressions

You can use any expression that contains variables and operators and evaluates to a natural number as an index:

In [15]:
i = 3; fruit[i+1]

'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)

In [16]:
fruit[end - 1]

'n': ASCII/Unicode U+006E (category Ll: Letter, lowercase)

In [17]:
length(fruit)

6

**length** is a built-in function that returns the number of characters in a string:

```Julia
	julia> fruits = "🍌 🍎 🍐"
		"🍌 🍎 🍐"

	julia> len = length(fruits)
		5
```

However, to get the last letter of a string, you might be tempted to try something like this:

```Julia
	julia> last = fruits[len]
		' ': ASCII/Unicode U+0020 (category Zs: Separator, space)
```

But you might not get what you expect.

In [18]:
fruits = "🍌 🍎 🍐"
sizeof(fruits)

14

### String indexing is byte based

UTF-8 is a **variable-width encoding**, meaning that not all characters are encoded in the same **number of bytes**.

The function **sizeof** gives the number of bytes in a string:

```Julia
	julia> sizeof("🍌")
		4
```

Because an emoji is encoded in 4 bytes and **string indexing is byte based**.

This means that not every byte index into a UTF-8 string is necessarily a valid index for a character. If you index into a string at such an invalid byte index, an error is thrown:

```Julia
	julia> fruits[2]
		ERROR: StringIndexError("🍌 🍎 🍐", 2)
```

In the case of fruits, the character 🍌 is a 4-byte character, so the indices 2, 3 and 4 are invalid and the next character’s index is 5; this next valid index can be computed with **nextind**, i.e., nextind(fruits, 1), and the next index after that by nextind(fruits, 5) and so on.

### String traversal

A lot of computations involve processing a string one character at a time. Often they start at the beginning, select each character in turn, do something with it, and continue until the end. This pattern of processing is called a **string traversal**. 

One way to write a traversal is with a while loop:

```Julia
	index = firstindex(fruits)
	while index <= sizeof(fruits)
		letter = fruits[index]
		println(letter)
		index = nextind(fruits, index)
	end
```

This loop traverses the string and displays each letter on a line by itself. The loop condition is index <= sizeof(fruit), so when index is larger than the number of bytes in the string, the condition is false, and the body of the loop does not run. The function **firstindex** returns the first valid byte index. 

In [23]:
function traverse(str)      # function definition
    index = firstindex(str)
    while index <= sizeof(str) 
        letter = str[index]
        print(letter, '\n')
        index = nextind(str, index)
    end
end


#---

fruits = "🍌 🍎 🍐 🍎 🍎 🍌"  # variable assignment or initialization

traverse(fruits)        # function call

🍌
 
🍎
 
🍐
 
🍎
 
🍎
 
🍌


### Exercise 8-1

Write a function that takes a string as an argument and displays the letters backward, one per line.

Another way to write a traversal is with a for loop and use the string as an **iterable** datatype:

In [24]:
for letter in fruits
    println(letter)
end

🍌
 
🍎
 
🍐
 
🍎
 
🍎
 
🍌


Each time through the loop, the next character in the string is assigned to the variable letter. The loop continues until no characters are left.

The following example shows how to use **concatenation (string multiplication)** and a for-loop to generate an abecedarian series (that is, in alphabetical order). 

In Robert McCloskey’s book **Make Way for Ducklings**, the names of the ducklings are Jack, Kack, Lack, Mack, Nack, Ouack, Pack, and Quack. This loop outputs these names in order:

```Julia
	prefixes = "JKLMNOPQ"
	suffix = "ack"

	for letter in prefixes
		println(letter * suffix)
	end

	Output:

	Jack
	Kack
	Lack
	Mack
	Nack
	Oack
	Pack
	Qack
```
Of course, that is not quite right because “Ouack” and “Quack” are misspelled.

### Exercise 8-2

Modify the program to fix this error.

### String slices

A segment of a sequence or a string is called a **slice**. Selecting a slice is similar to selecting a character:

```Julia
	julia> str = "Julius Caesar";

	julia> str[1:6]
		"Julius"
```

The **operator [n:m]** returns the part of the string **from the n-th byte to the m-th byte**. So the same caution is needed as for simple indexing. The **end** keyword can be used to indicate the last byte of the string:

```Julia
	julia> str[8:end]
		"Caesar"
```

If the first index is greater than the second the result is an **empty string**, represented by two quotation marks:

```Julia
	julia> str[8:7]
		""
```

An empty string contains no characters and has length 0, but other than that, it is the same as any other string.

### Exercise 8-3

Continuing this example, what do you think str[:] means? Try it and see.

### Strings are immutable

It is tempting to use the [ ] operator on the left side of an assignment, with the intention of changing a character in a string. For example:

In [25]:
greeting = "Hello, world!"
greeting[1] = 'J'

MethodError: MethodError: no method matching setindex!(::String, ::Char, ::Int64)

The reason for the error is that strings are **immutable**, which means you cannot change an existing string. **The best you can do is create a new string that is a variation on the original**:

In [26]:
greetingJ = "J" * greeting[2:end]

"Jello, world!"

This example concatenates a new first letter onto a slice of greeting. It has no effect on the original string.

### String interpolation

Constructing strings using concatenation can become a bit cumbersome. To reduce the need for these verbose calls to strings or repeated multiplications, Julia allows **string interpolation using $**:

```Julia
	julia> greet = "Hello"
		"Hello"

	julia> whom = "World"
		"World"

	julia> "$greet, $(whom)!"
		"Hello, World!"
```

This is more readable and convenient than string concatenation: 

```Julia
	julia> greet * ", " * whom * "!"
```

The **shortest complete expression after the $** is taken as the expression whose value is to be interpolated into the string. Thus, you can interpolate any expression into a string using parentheses:

```Julia
	julia> "1 + 2 = $(1 + 2)"
		"1 + 2 = 3"
```

### String search

What does the following function do?

```Julia
	function find(word::String, letter::Char)
		index = firstindex(word)
		while index <= sizeof(word)
			if word[index] == letter
				return index
			end
			index = nextind(word, index)
		end
		return -1
	end
```

In a sense, `find` is the inverse of the `bracket [] operator`. Instead of taking an index and extracting the corresponding character, it takes a character and finds the first index where that character appears in the string word. If the character is not found, the function returns -1.

This is the first example we have seen of a return statement inside a loop. 

If word[index] == letter, the function breaks out of the loop and returns immediately. If the character does not appear in the string, the program exits the loop normally and returns -1.

This pattern of computation—traversing a sequence and returning when we find what we are looking for—is called a **search**.

### Exercise 8-4

Modify find so that it has a third parameter, the index in word where it should start looking.

### Looping and counting

The following program counts the number of times the letter a appears in a string:

```Julia
	word = "banana"
	counter = 0

	for letter in word
		if letter == 'a'
			counter += 1
		end
	end

	println(counter)
```

This program demonstrates another pattern of computation called a **counter**. 

The variable counter is initialized to 0 and then incremented each time an `'a'` is found. When the loop exits, counter contains the result—the total number of a’s in the string.

### Exercise 8-5

Encapsulate this code in a function named count, and generalize it so that it accepts the string and the letter as arguments. Then rewrite the function so that instead of traversing the string, it uses the three-parameter version of find from the previous section.

### String library

Julia provides functions that perform a variety of useful operations on strings. For example, the function **uppercase** takes a string and returns a new string with all uppercase letters.

```Julia
	julia> uppercase("Hello, World!")
		"HELLO, WORLD!"
```

As it turns out, there is a function named **findfirst** that is remarkably similar to the function find we wrote:

```Julia
	julia> findfirst("a", "banana")
		2:2
```

Actually, the findfirst function is more general than our function; it can find **substrings**, not just characters:

```Julia
	julia> findfirst("na", "banana")
		3:4
```

By default, findfirst starts at the beginning of the string, but the function **findnext** takes a third argument, the index where it should start:

```Julia
	julia> findnext("na", "banana", 4)
		5:6
```

### The _in_ or _∈_ Operator

The operator **∈** (\in<TAB>) is a boolean operator that takes a character and a string and returns true if the first appears in the second:

```Julia
	julia> 'a' ∈ "banana"    # 'a' in "banana"
		true
```

For example, the following function prints all the letters from word1 that also appear in word2:

In [27]:
function inboth(word1, word2)	
    for letter in word1
        if letter in word2
            print(letter, " ")
        end
    end
end

#---

inboth("apples and oranges", "see below")

l e s     o e s 

### String comparison

The relational operators work on strings. To see if two strings are equal:

```Julia
	word = "Pineapple"
	if word == "banana"
		println("All right, bananas.")
	end
```

Other relational operations are useful for putting words in alphabetical order:

In [28]:
function word_comparison(word)
    if word < "banana"
        println("Your word, $word, comes before banana.")
    elseif word > "banana"
        println("Your word, $word, comes after banana.")
    else
        println("All right, bananas.")
    end
end

word_comparison("banAna")

Your word, banAna, comes before banana.


Julia does not handle uppercase and lowercase letters the same way people do. All the uppercase letters come before all the lowercase letters, so: Your word, Pineapple, comes before banana. A common way to address this problem is to convert strings to a standard format, such as all lowercase, before performing the comparison.

### Debugging

When you use indices to traverse the values in a sequence, it is tricky to get the beginning and end of the traversal right. Here is a function that is supposed to compare two words and return true if one of the words is the reverse of the other, but it contains two errors:

```Julia
	function isreverse(word1::String, word2::String)
		if length(word1) != length(word2)
			return false
		end

		i = firstindex(word1)
		j = lastindex(word2)
		while j >= 0
			j = prevind(word2, j)
			if word1[i] != word2[j]
				return false
			end
			i = nextind(word1, i)
		end
		return true
	end
```

The first if statement checks whether the words are the same length. If not, we can return false immediately. Otherwise, for the rest of the function, we can assume that the words are the same length. This is an example of the **guardian pattern**.

i and j are indices: i traverses word1 forward while j traverses word2 backward. If we find two letters that do not match, we can return false immediately. If we get through the whole loop and all the letters match, we return true.

The function **lastindex** returns the last valid byte index of a string and **prevind** the previous valid index of a character.

If we test this function with the words "pots" and "stop", we expect the return value true, but we get false:

```Julia
	julia> isreverse("pots", "stop")
		false
```

For debugging this kind of error, my first move is to print the values of the indices:

```Julia
	while j >= 0
		j = prevind(word2, j)
		@show i j
		if word1[i] != word2[j]
```

Now when I run the program again, I get more information:

```Julia
	julia> isreverse("pots", "stop")
		i = 1
		j = 3
		false
```

The first time through the loop, the value of j is 3 but it should be 4. This can be fixed by moving j = prevind(word2, j) to the end of the while loop.

```Julia
	function isreverse(word1::String, word2::String)
		if length(word1) != length(word2)
			return false
		end

		i = firstindex(word1)
		j = lastindex(word2)
		while j >= 0
			@show i j
			if word1[i] != word2[j]
				return false
			end
			i = nextind(word1, i)
			j = prevind(word2, j)	#  moving j = prevind(word2, j) here
		end
		return true
	end
```

If I fix that error and run the program again, I get:

```Julia
	julia> isreverse("pots", "stop")
		i = 1
		j = 4
		i = 2
		j = 3
		i = 3
		j = 2
		i = 4
		j = 1
		i = 5
		j = 0
	ERROR: BoundsError: attempt to access "pots"
		at index [5]
```

### Exercise 8-6

Find and fix the second error in this function.

## Exercises

### Exercise 8-7

Read the documentation of the string functions at https://docs.julialang.org/en/v1/manual/strings/. 

You might want to experiment with some of them to make sure you understand how they work. **strip** and **replace** are particularly useful. The documentation uses a syntax that might be confusing. For example, in 

```Julia
	search(string::AbstractString, chars::Chars, [start::Integer])
```

the brackets indicate **optional arguments** and **::** represent a type specification. So string and chars are required, but start is optional.

### Exercise 8-8

There is a built-in function called `count` that is similar to the function in Looping and Counting. Read the documentation of this function and use it to count the number of a’s in "banana".

### Exercise 8-9

A string slice can take a third index. The first specifies the start, the third the end and the second the **step size**; that is, the number of spaces between successive characters. A step size of 2 means every other character; 3 means every third, etc.

```Julia
	julia> fruit = "banana"
	"banana"

	julia> fruit[1:2:6]
	"bnn"
```

A step size of -1 goes through the word backwards, so the slice **[end:-1:1]** generates a reversed string. Use this idiom to write a one-line version of ispalindrome from Exercise 6-6.


In [29]:
function ispalindrome(str)
	return str[end:-1:1] == str
end

ispalindrome("apa")

true

### Exercise 8-10

The following functions are all intended to check whether a string contains any lowercase letters, but at least some of them are wrong. For each function, describe what the function actually does (assuming that the parameter is a string).

```Julia
	function anylowercase1(s)
		for c in s
			if islowercase(c)
				return true
			else
				return false
			end
		end
	end
```

```Julia
	function anylowercase2(s)
		for c in s
			if islowercase('c')
				return "true"
			else
				return "false"
			end
		end
	end
```

```Julia
	function anylowercase3(s)
		for c in s
			flag = islowercase(c)
		end
		flag
	end
```

```Julia
	function anylowercase4(s)
		flag = false
		for c in s
			flag = flag || islowercase(c)
		end
		flag
	end
```

```Julia
	function anylowercase5(s)
		for c in s
			if !islowercase(c)
				return false
			end
		end
		true
	end
```

### Exercise 8-11

A Caesar cypher is a weak form of encryption that involves “rotating” each letter by a fixed number of places. To rotate a letter means to shift it through the alphabet, wrapping around to the beginning if necessary, so ’A’ rotated by 3 is ’D’ and ’Z’ rotated by 1 is ’A’. To rotate a word, rotate each letter by the same amount. For example, "cheer" rotated by 7 is "jolly".

Write a function called rotateword that takes a string and an integer as parameters, and returns a new string that contains the letters from the original string rotated by the given amount.

You might want to use the built-in function **Int**, which converts a character to a numeric code, and Char, which converts numeric codes to characters. Letters of the alphabet are encoded in alphabetical order, so for example:

```Julia
	julia> Int('c') - Int('a')
		2
```

Because c is the third letter of the alphabet. But beware: the numeric codes for uppercase letters are different.

```Julia
	julia> Char(Int('A') + 32)
		'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
```